# [CPSC 322](https://github.com/GonzagaCPSC322) Data Science Algorithms
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Flask
What are our learning objectives for this lesson?
* Learn about web frameworks
* Use flask to create a simple web app
    * Utilize routes, debug mode, `request` objects, and query string arguments
    * Return HTML and JSON responses to get requests
* Learn about HTTP status codes

Content used in this lesson is based upon information in the following sources:
* [Flask](https://flask.palletsprojects.com/en/1.1.x/)
* Flask Web Development by Miguel Grinberg

## Warm-up Task(s)
1. Please answer the morning question at http://PollEv.com/ginasprint096
1. Go to https://heroku.com/ and make a free account
1. In your Docker container's bash shell, run `apt install gnupg` and `curl https://cli-assets.heroku.com/install-ubuntu.sh | sh` to install the Heroku CLI
1. Open APIServiceFun, we are going to start adding "routes"!!
    1. Note: if you don't use VS Code attached to the container for development in this class, you will need to create a new anaconda3 container with a binding for port 5000 if you want to follow along today (see Docker cheat sheet for updated `docker run` command)

## Today
* Announcements
    * RQ9 is due on Monday
    * PA6 is due next week. Questions?
    * IQ8 on Thursday on decision trees and entropy (see [B Attribute Selection lab tasks 5 and 6](https://github.com/GonzagaCPSC322/U5-Decision-Trees/blob/master/B%20Attribute%20Selection.ipynb) for additional entropy practice)
* Flask apps and deployment to Heroku
* ~10 mins of class: talk about the project (the Project is posted in our 322 GDrive)

## Introduction to Flask
[Flask](https://flask.palletsprojects.com/en/1.1.x/) is a micro web framework. From [Wikipedia](https://en.wikipedia.org/wiki/Web_framework):
>A web framework (WF) or web application framework (WAF) is a software framework that is designed to support the development of web applications including web services, web resources, and web APIs. Web frameworks provide a standard way to build and deploy web applications on the World Wide Web. Web frameworks aim to automate the overhead associated with common activities performed in web development. For example, many web frameworks provide libraries for database access, templating frameworks, and session management, and they often promote code reuse.[1] Although they often target development of dynamic web sites, they are also applicable to static websites.[2]

We can use Flask to create a web app that either runs locally or runs on a server. In this class, we want to build a web app that makes predictions for unseen test instances. The app could be used directly (e.g. go to its URL) or could be used by a client via its API (application programming interface) (e.g. an iOS app that uses the web app's API to get a prediction).

A basic Flask app has the following structure:

```python
from flask import Flask
app = Flask(__name__)

@app.route("/")
def index():
    return "<h1>Hello World!</h1>"

if __name__ == "__main__":
    app.run()
```

This creates a Flask app with a single ["route"](https://flask.palletsprojects.com/en/1.1.x/quickstart/#routing), which is a path on the server to a function that handles it. This route is to the main index page of the web app. By default, a request to a route is assumed to be a GET request. We can make this explicit with `@app.route("/", methods=["GET"])`. For a GET request, the function should return something. The something can be a JSON response (like an API) or a HTML (for the client to render in their web browser). This simple app returns HTML with paragraph text that reads "Hello World!".

To run this app, create an `app.py` and copy the code above into it. Run `python app.py` then go to your web browser and paste `http://127.0.0.1:5000/` in the URL bar. You should see Hello World! This is because your browser made a GET request to your Flask app that is running locally (on your machine only, not on the actual web) and listening for requests on port 5000 (the default port Flask runs on). The `index()` function executed because of the `/` route on the end of the url.

You can hit CTRL + C at your keyboard to kill the Flask server.

Note: the Flask server that is running with `app.run()` is a *development* server, meaning it is not as robust as a *production* sever should be. Therefore, it is discouraged from being used in production. For large production projects, it is considered best practice to use a production level server like [WSGI](https://www.fullstackpython.com/wsgi-servers.html) or [Gunicorn](https://gunicorn.org/) instead. 

## Debug Mode
By default, `app.run()` does not start the Flask web server in debug mode. For debugging purposes, it is helpful to turn debug mode on with `app.run(debug=True)`. When this flag is set to `True`, the reloader is enabled (allowing your Flask app to automatically reload whenever a source file in your project is modified (super handy!). It also enables the debugger, which will show an interactive stack trace in the browser window when your application raises an unhandled exception. 

Note: never deploy a Flask app with debug enabled!!

## Request Arguments
Often URLs contain query strings which contain details about the client's request. For example, the following iTunes Search request URL has two query arguments in its query string (the string after the ?): https://itunes.apple.com/search?term=shrek&media=movie The two arguments are the key-value pairs term=shrek and media=movie. The /search route maps to a function that extracts these two key value pairs and uses them to query a database and prepare a response. In Flask, we can access query arguments from a query string by accessing the [`request` object](https://flask.palletsprojects.com/en/1.1.x/quickstart/#accessing-request-data) that is in the scope of a route function. To access parameters submitted in the URL (?key=value) you can use the `args` attribute: `search_term = request.args.get("term", "")` With this call to `get()`, "" will be returned if the key is not found in the query string. 

Here is a hello route that returns HTML with paragraph text from `"Hello, <name>"` where `<name>` is extracted from a query string:

```python
from flask import request
# ...

@app.route("/hello", methods=["GET"])
def hello():
    name = request.args.get("name", "unknown")
    return "<h1>Hello, {}!</h1>".format(name)
```

To test this route, go to `http://127.0.0.1:5000/hello?name=spike` in your web browser. You should see "Hello, spike!". Try putting your name in place of spike in the query string and see what the function returns!

## Returning JSON Instead of HTML
So far, our routes have handled GET requests by returning HTML, assuming our clients want to render this HTML in a web page. What if we want to have an API endpoint that returns JSON data instead? We can return a different response from a route function that is JSON data. Then the client can parse the JSON response and grab the data it wants (just like how we parsed the iTunes Search API request response). 

Here is an add route that returns JSON with the result of adding two integer numbers from the query string together:

```python
from flask import jsonify
# ...

@app.route('/add', methods=["GET"])
def add():
    arg1 = request.args.get('arg1')
    arg2 = request.args.get('arg2')
    
    result = do_computation(arg1, arg2)
    if isinstance(result, int):
        # success!
        result = {"result": result}
        return jsonify(result), 200
    else:
        return "Arguments were not integers", 400

def do_computation(arg1, arg2):
    try:
        arg1 = int(arg1)
        arg2 = int(arg2)
        return arg1 + arg2
    except:
        return "Arguments were not integers"
```

The function grabs two arguments from the `request` object and tries to add them together as integers in `do_computation()`. If the add is successful, an integer is returned. Otherwise, an error string is returned. If an integer is returned from `do_computation()`, it is packaged into a dictionary and "jsonified" into a JSON object. This object is returned with the HTTP status code 200, which represents "OK" (the request has succeeded). If an integer is not returned from `do_computation()`, the error string is returned with the HTTP status code 400, which represents "Bad Request" (the server could not understand the request due to invalid syntax). To learn more about status codes, [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) is a full list of HTTP status codes from Mozilla Developer.